In [1]:
%load_ext autoreload
%autoreload
from IPython.display import clear_output

import os
import sys
import subprocess
import pdb
import time
import datetime
import math
import random
import _pickle as cPickle
from collections import defaultdict

from six.moves import zip_longest
import numpy as np
import pandas as pd

import tensorflow as tf
from tensorflow import distributions as tfd
from tensorflow.keras.preprocessing.sequence import pad_sequences

from data_structure import get_batches, get_test_batches
from components import tf_log, sample_latents, compute_kl_loss, dynamic_rnn, dynamic_bi_rnn
from topic_model import TopicModel

from topic_beam_search_decoder import BeamSearchDecoder

In [2]:
PAD = '<pad>' # This has a vocab id, which is used to pad the encoder input, decoder input and target sequence
UNK = '<unk>' # This has a vocab id, which is used to represent out-of-vocabulary words
BOS = '<p>' # This has a vocab id, which is used at the beginning of every decoder input sequence
EOS = '</p>' # This has a vocab id, which is used at the end of untruncated target sequences

# load data & set config

In [3]:
def del_all_flags(FLAGS):
    flags_dict = FLAGS._flags()    
    keys_list = [keys for keys in flags_dict]    
    for keys in keys_list:
        FLAGS.__delattr__(keys)

del_all_flags(tf.flags.FLAGS)

flags = tf.app.flags

flags.DEFINE_string('gpu', '3', 'visible gpu')

flags.DEFINE_string('mode', 'train', 'set train or eval')

flags.DEFINE_string('data_path', 'data/bags/instances.pkl', 'path of data')
flags.DEFINE_string('modeldir', 'model/tglm_vae_tmp2', 'directory of model')
flags.DEFINE_string('modelname', 'bags', 'name of model')

flags.DEFINE_integer('epochs', 50, 'epochs')
flags.DEFINE_integer('batch_size', 64, 'number of sentences in each batch')
flags.DEFINE_integer('log_period', 500, 'valid period')

flags.DEFINE_string('opt', 'Adagrad', 'optimizer')
flags.DEFINE_float('lr', 0.1, 'lr')
flags.DEFINE_float('reg', 1., 'regularization term')
flags.DEFINE_float('grad_clip', 5., 'grad_clip')

flags.DEFINE_float('keep_prob', 0.8, 'dropout rate')
flags.DEFINE_float('word_keep_prob', 0.75, 'word dropout rate')

flags.DEFINE_bool('warmup', True, 'flg of warming up')
flags.DEFINE_integer('epochs_cycle', 5, 'number of epochs within a cycle')
flags.DEFINE_float('r_cycle', 0.5, 'proportion used to increase beta within a cycle')
flags.DEFINE_integer('warmup_topic', 0, 'warmup period for KL of topic')

flags.DEFINE_integer('beam_width', 2, 'beam_width')
flags.DEFINE_float('length_penalty_weight', 0.0, 'length_penalty_weight')

flags.DEFINE_integer('n_topic', 50, 'number of topic')
flags.DEFINE_integer('dim_hidden_bow', 256, 'dim of hidden bow')
flags.DEFINE_integer('dim_latent_bow', 32, 'dim of latent topic')
flags.DEFINE_integer('dim_emb', 256, 'dim_emb')
flags.DEFINE_integer('dim_hidden', 512, 'dim_hidden')
flags.DEFINE_integer('dim_hidden_topic', 512, 'dim_hidden_topic')
flags.DEFINE_integer('dim_latent', 32, 'dim_latent')
flags.DEFINE_bool('bidirectional', True, 'flg of bidirectional encoding')

# for evaluation
flags.DEFINE_string('refdir', 'ref', 'refdir')
flags.DEFINE_string('outdir', 'out', 'outdir')

flags.DEFINE_string('f', '', 'kernel')
flags.DEFINE_bool('logtostderr', True, 'kernel')
flags.DEFINE_bool('showprefixforinfo', False, '')
flags.DEFINE_bool('verbosity', False, '')
# flags.DEFINE_integer('stderrthreshold', 20, 'kernel')

config = flags.FLAGS

flags.DEFINE_string('modelpath', os.path.join(config.modeldir, config.modelname), 'path of model')

In [4]:
os.environ["CUDA_VISIBLE_DEVICES"] = config.gpu

In [5]:
instances_train, instances_dev, instances_test, word_to_idx, idx_to_word, bow_idxs = cPickle.load(open(config.data_path,'rb'))

In [6]:
train_batches = get_batches(instances_train, config.batch_size)
dev_batches = get_batches(instances_dev, config.batch_size)
test_batches = get_test_batches(instances_test, config.batch_size)

In [7]:
flags.DEFINE_integer('PAD_IDX', word_to_idx[PAD], 'PAD_IDX')
flags.DEFINE_integer('UNK_IDX', word_to_idx[UNK], 'UNK_IDX')
flags.DEFINE_integer('BOS_IDX', word_to_idx[BOS], 'BOS_IDX')
flags.DEFINE_integer('EOS_IDX', word_to_idx[EOS], 'EOS_IDX')

flags.DEFINE_integer('n_vocab', len(word_to_idx), 'n_vocab')
flags.DEFINE_integer('dim_bow', len(bow_idxs), 'dim_bow')

maximum_iterations = max([max([instance.max_sent_l for instance in batch]) for ct, batch in dev_batches])
flags.DEFINE_integer('maximum_iterations', maximum_iterations, 'maximum_iterations')

flags.DEFINE_integer('cycle_steps', len(train_batches)*config.epochs_cycle, 'number of steps for each cycle')

# build language model 

## feed dict

In [8]:
tf.reset_default_graph()

t_variables = {}
t_variables['bow'] = tf.placeholder(tf.float32, [None, config.dim_bow], name='bow')
t_variables['input_token_idxs'] = tf.placeholder(tf.int32, [None, None], name='input_token_idxs')
t_variables['dec_input_idxs'] = tf.placeholder(tf.int32, [None, None], name='dec_input_idxs')
t_variables['dec_target_idxs'] = tf.placeholder(tf.int32, [None, None], name='dec_target_idxs')
t_variables['batch_l'] = tf.placeholder(tf.int32, name='batch_l')
t_variables['doc_l'] = tf.placeholder(tf.int32, [None], name='doc_l')
t_variables['sent_l'] = tf.placeholder(tf.int32, [None], name='sent_l')
t_variables['keep_prob'] = tf.placeholder(tf.float32, name='keep_prob')

In [9]:
def get_feed_dict(batch, mode='train', assertion=False):
    def token_dropout(sent_idxs):
        sent_idxs_dropout = np.asarray(sent_idxs)
        sent_idxs_dropout[np.random.rand(len(sent_idxs)) > config.word_keep_prob] = config.UNK_IDX
        return list(sent_idxs_dropout)

    bow = np.array([instance.bow for instance in batch]).astype(np.float32)
    
    doc_l = np.array([len(instance.token_idxs) for instance in batch])
    
    feed_input_token_idxs_list = [sent_idxs for instance in batch for sent_idxs in instance.token_idxs]
    feed_dec_input_idxs_list = [[config.BOS_IDX] + token_dropout(sent_idxs) for sent_idxs in feed_input_token_idxs_list]
    feed_dec_target_idxs_list = [sent_idxs + [config.EOS_IDX]  for sent_idxs in feed_input_token_idxs_list]
        
    sent_l = np.array([len(sent_idxs) for sent_idxs in feed_input_token_idxs_list], np.int32)
    batch_l = len(sent_l)
    
    feed_input_token_idxs = pad_sequences(feed_input_token_idxs_list, padding='post', value=config.PAD_IDX, dtype=np.int32)
    feed_dec_input_idxs = pad_sequences(feed_dec_input_idxs_list, padding='post', value=config.PAD_IDX, dtype=np.int32)
    feed_dec_target_idxs = pad_sequences(feed_dec_target_idxs_list, padding='post', value=config.PAD_IDX, dtype=np.int32)
    
    if assertion:
        index = 0
        for instance in batch:
            for line_idxs in instance.token_idxs:
                assert feed_input_token_idxs_list[index] == line_idxs
                index += 1
        assert feed_input_token_idxs.shape[1] == np.max(sent_l)
        assert feed_dec_input_idxs.shape[1] == np.max(sent_l) + 1
        assert feed_dec_target_idxs.shape[1] == np.max(sent_l) + 1
    
    keep_prob = config.keep_prob if mode == 'train' else 1.0

    feed_dict = {
                t_variables['bow']: bow, 
                t_variables['batch_l']: batch_l, t_variables['doc_l']: doc_l, t_variables['sent_l']: sent_l, 
                t_variables['input_token_idxs']: feed_input_token_idxs, t_variables['dec_input_idxs']: feed_dec_input_idxs, t_variables['dec_target_idxs']: feed_dec_target_idxs, 
                t_variables['keep_prob']: keep_prob
    }
    return  feed_dict

In [10]:
def debug_shape(variables):
    sample_batch = dev_batches[0][1]
    feed_dict = get_feed_dict(sample_batch)
    _variables = sess.run(variables, feed_dict=feed_dict)
    for _variable, variable in zip(_variables, variables):
        if hasattr(variable, 'name'):
            print(variable.name, ':', _variable.shape)
        else:
            print(_variable.shape)

def debug_value(variables, return_value=False):
    sample_batch = test_batches[0][1]
    feed_dict = get_feed_dict(sample_batch)
    _variables = sess.run(variables, feed_dict=feed_dict)

    if return_value: 
        return _variables
    else:
        for _variable, variable in zip(_variables, variables):
            if hasattr(variable, 'name'):
                print(variable.name, ':', _variable)
            else:
                print(_variable)
                
def check_shape(variables):
    if 'sess' in globals(): raise
    sess = tf.Session()
    sess.run(tf.global_variables_initializer())
    
    sample_batch = test_batches[0][1]
    feed_dict = get_feed_dict(sample_batch)
    _variables = sess.run(variables, feed_dict=feed_dict)
    for _variable, variable in zip(_variables, variables):
        if hasattr(variable, 'name'):
            print(variable.name, ':', _variable.shape)
        else:
            print(_variable.shape)
            
    sess.close()
    
def check_value(variables):
    if 'sess' in globals(): raise
    sess = tf.Session()
    sess.run(tf.global_variables_initializer())
    
    sample_batch = test_batches[0][1]
    feed_dict = get_feed_dict(sample_batch)
    _variables = sess.run(variables, feed_dict=feed_dict)
    for _variable, variable in zip(_variables, variables):
        if hasattr(variable, 'name'):
            print(variable.name, ':', _variable)
        else:
            print(_variable.shape)
            
    sess.close()    
    
# sent_loss_kl_categ_tmp = tf.reduce_mean(tf.reduce_sum(tf.multiply(prob_topic_infer, tf_log(prob_topic_infer/prob_topic_sents)), 1))
# debug_value([sent_loss_kl_categ, sent_loss_kl_categ_tmp])
# sent_loss_kl_gauss_tmp = 0.5 * tf.reduce_sum(tf.exp(logvars_topic_infer-logvars_topic) + tf.square(means_topic - means_topic_infer) / tf.exp(logvars_topic) - 1 + (logvars_topic - logvars_topic_infer), -1)
# sent_loss_kl_gmm_tmp = tf.reduce_mean(tf.reduce_sum(tf.multiply(prob_topic_infer, sent_loss_kl_gauss_tmp), -1))
# debug_value([sent_loss_kl_gmm_tmp, sent_loss_kl_gmm])    

## topic model

In [11]:
# encode bow
with tf.variable_scope('topic/enc', reuse=False):
    hidden_bow_ = tf.layers.Dense(units=config.dim_hidden_bow, activation=tf.nn.relu, name='hidden_bow')(t_variables['bow'])
    hidden_bow = tf.layers.Dropout(t_variables['keep_prob'])(hidden_bow_)
    means_bow = tf.layers.Dense(units=config.dim_latent_bow, name='mean_bow')(hidden_bow)
    logvars_bow = tf.layers.Dense(units=config.dim_latent_bow, kernel_initializer=tf.constant_initializer(0), bias_initializer=tf.constant_initializer(0), name='logvar_topic')(hidden_bow)
    latents_bow = sample_latents(means_bow, logvars_bow) # sample latent vectors

    prob_topic = tf.layers.Dense(units=config.n_topic, activation=tf.nn.softmax, name='prob_topic')(latents_bow) # inference of topic probabilities

# decode bow
with tf.variable_scope('shared', reuse=False):
    embeddings = tf.get_variable('emb', [config.n_vocab, config.dim_emb], dtype=tf.float32, initializer=tf.contrib.layers.xavier_initializer()) # embeddings of vocab

bow_embeddings = tf.nn.embedding_lookup(embeddings, bow_idxs) # embeddings of each bow features

with tf.variable_scope('topic/dec', reuse=False):
    topic_embeddings = tf.get_variable('topic_emb', [config.n_topic, config.dim_emb], dtype=tf.float32, initializer=tf.contrib.layers.xavier_initializer()) # embeddings of topics

    topic_bow = tf.nn.softmax(tf.matmul(topic_embeddings, bow_embeddings, transpose_b=True), 1) # bow vectors for each topic
    logits_bow = tf_log(tf.matmul(prob_topic, topic_bow)) # predicted bow distribution

    # prior of each gaussian distribution (computed for each topic)
    hidden_topic = tf.layers.Dense(units=config.dim_hidden_topic, activation=tf.nn.relu, name='hidden_topic')(topic_bow)
    means_topic = tf.layers.Dense(units=config.dim_latent, name='mean_topic')(hidden_topic)
    logvars_topic = tf.layers.Dense(units=config.dim_latent, kernel_initializer=tf.constant_initializer(0), bias_initializer=tf.constant_initializer(0), name='logvar_topic')(hidden_topic)
    sigma_topic = tf.exp(0.5 * logvars_topic)
    gauss_topic = tfd.Normal(loc=means_topic, scale=sigma_topic)    
    
# define losses
topic_losses_recon = -tf.reduce_sum(tf.multiply(t_variables['bow'], logits_bow), 1)
topic_loss_recon = tf.reduce_mean(topic_losses_recon) # negative log likelihood of each words

topic_loss_kl = compute_kl_loss(means_bow, logvars_bow) # KL divergence b/w latent dist & gaussian std

topic_bow_norm = topic_bow / tf.norm(topic_bow, axis=1, keepdims=True)
topic_dots = tf.clip_by_value(tf.matmul(topic_bow_norm, tf.transpose(topic_bow_norm)), -1., 1.)
topic_loss_reg = tf.reduce_mean(tf.square(topic_dots - tf.eye(config.n_topic)))
# topic_angles = tf.acos(topic_dots)
# topic_angles_mean = tf.reduce_mean(topic_angles)
# topic_angles_vars = tf.reduce_mean(tf.square(topic_angles - topic_angles_mean))
# topic_loss_reg = tf.exp(topic_angles_vars - topic_angles_mean)

# monitor
n_bow = tf.reduce_sum(t_variables['bow'], 1)
topic_ppls = tf.divide(topic_losses_recon, tf.maximum(1e-5, n_bow))
topics_freq_bow_indices = tf.nn.top_k(topic_bow, 10, name='topic_freq_bow').indices

## encoder

In [12]:
# input
input_token_idxs = t_variables['input_token_idxs']
batch_l = t_variables['batch_l']
sent_l = t_variables['sent_l']
max_sent_l = tf.reduce_max(sent_l)

with tf.variable_scope('sent/enc', reuse=False):
    # get word embedding
    enc_input = tf.nn.embedding_lookup(embeddings, input_token_idxs)

    # get sentence embedding
    _, enc_state = dynamic_bi_rnn(enc_input, sent_l, config.dim_hidden, t_variables['keep_prob'])

    # TODO House Holder flow
    hidden_topic_infer =  tf.layers.Dense(units=config.dim_hidden, activation=tf.nn.relu, name='hidden_topic_infer')(enc_state)
    prob_topic_infer = tf.layers.Dense(units=config.n_topic, activation=tf.nn.softmax, name='prob_topic_infer')(hidden_topic_infer)

    w_mean_topic_infer = tf.get_variable('mean_topic_infer/kernel', [config.n_topic, enc_state.shape[-1], config.dim_latent], dtype=tf.float32)
    b_mean_topic_infer = tf.get_variable('mean_topic_infer/bias', [1, config.n_topic, config.dim_latent], dtype=tf.float32)
    means_topic_infer = tf.tensordot(enc_state, w_mean_topic_infer, axes=[[1], [1]]) + b_mean_topic_infer
    
    w_logvar_topic_infer = tf.get_variable('logvar_topic_infer/kernel', [config.n_topic, enc_state.shape[-1], config.dim_latent], dtype=tf.float32, initializer=tf.constant_initializer(0))
    b_logvar_topic_infer = tf.get_variable('logvar_topic_infer/bias', [1, config.n_topic, config.dim_latent], dtype=tf.float32, initializer=tf.constant_initializer(0))
    logvars_topic_infer = tf.tensordot(enc_state, w_logvar_topic_infer, axes=[[1], [1]]) + b_logvar_topic_infer
    sigma_topic_infer = tf.exp(0.5 * logvars_topic_infer)
    gauss_topic_infer = tfd.Normal(loc=means_topic_infer, scale=sigma_topic_infer)
    
    # latent vectors from each gaussian dist.
    latents_topic_infer = sample_latents(means_topic_infer, logvars_topic_infer) 
    # latent vector from gaussian mixture
    latents_input = tf.matmul(tf.expand_dims(prob_topic_infer, -1), latents_topic_infer, transpose_a=True)
    
    # for beam search
    means_input = tf.matmul(tf.expand_dims(prob_topic_infer, -1), means_topic_infer, transpose_a=True)    

## decoder

In [13]:
# prepare for decoding
dec_sent_l = tf.add(sent_l, 1)
dec_input_idxs = t_variables['dec_input_idxs']
dec_input = tf.nn.embedding_lookup(embeddings, dec_input_idxs)

dec_latents_input = tf.tile(latents_input, [1, tf.shape(dec_input)[1], 1])
dec_concat_input = tf.concat([dec_input, dec_latents_input], -1)

# decode for training
with tf.variable_scope('sent/dec/rnn', initializer=tf.contrib.layers.xavier_initializer(), dtype = tf.float32, reuse=False):
    dec_cell = tf.contrib.rnn.GRUCell(config.dim_hidden)
    dec_cell = tf.contrib.rnn.DropoutWrapper(dec_cell, output_keep_prob = t_variables['keep_prob'])

    dec_initial_state = tf.layers.Dense(units=config.dim_hidden, activation=tf.nn.relu, name='init_state')(tf.squeeze(latents_input, 1))
    
    helper = tf.contrib.seq2seq.TrainingHelper(inputs=dec_concat_input, sequence_length=dec_sent_l)

    train_decoder = tf.contrib.seq2seq.BasicDecoder(
        cell=dec_cell,
        helper=helper,
        initial_state=dec_initial_state)

    dec_outputs, _, output_sent_l = tf.contrib.seq2seq.dynamic_decode(train_decoder)
    
    output_layer = tf.layers.Dense(config.n_vocab, use_bias=False, name='out')
    output_logits = output_layer(dec_outputs.rnn_output)
    
    output_token_idxs = tf.argmax(output_logits, 2)

In [14]:
start_tokens = tf.fill([batch_l], config.BOS_IDX)
end_token = config.EOS_IDX

with tf.variable_scope('sent/dec/rnn', reuse=True):
    infer_dec_initial_state = tf.layers.Dense(units=config.dim_hidden, activation=tf.nn.relu, name='init_state')(tf.squeeze(means_input, 1))
    beam_dec_initial_state = tf.contrib.seq2seq.tile_batch(infer_dec_initial_state, multiplier=config.beam_width)
    beam_latents_input = tf.contrib.seq2seq.tile_batch(tf.squeeze(means_input, 1), multiplier=config.beam_width) # added
    
    beam_decoder = BeamSearchDecoder(
        cell=dec_cell,
        embedding=embeddings,
        start_tokens=start_tokens,
        end_token=end_token,
        initial_state=beam_dec_initial_state,
        beam_width=config.beam_width, 
        output_layer=output_layer,
        length_penalty_weight=config.length_penalty_weight,
        latents_input=beam_latents_input)

    beam_dec_outputs, _, _ = tf.contrib.seq2seq.dynamic_decode(
        beam_decoder,
        maximum_iterations = config.maximum_iterations)

    beam_output_token_idxs = beam_dec_outputs.predicted_ids[:, :, 0]

In [15]:
with tf.variable_scope('sent/dec/rnn', reuse=True):
    inter_means_input = tf.placeholder(tf.float32, [None, config.dim_latent])
    
    inter_dec_initial_state = tf.layers.Dense(units=config.dim_hidden, activation=tf.nn.relu, name='init_state')(inter_means_input)
    inter_beam_dec_initial_state = tf.contrib.seq2seq.tile_batch(inter_dec_initial_state, multiplier=config.beam_width)
    inter_beam_latents_input = tf.contrib.seq2seq.tile_batch(inter_means_input, multiplier=config.beam_width) # added
    
    inter_beam_decoder = BeamSearchDecoder(
        cell=dec_cell,
        embedding=embeddings,
        start_tokens=start_tokens,
        end_token=end_token,
        initial_state=inter_beam_dec_initial_state,
        beam_width=config.beam_width, 
        output_layer=output_layer,
        length_penalty_weight=config.length_penalty_weight,
        latents_input=inter_beam_latents_input)

    inter_beam_dec_outputs, _, _ = tf.contrib.seq2seq.dynamic_decode(
        inter_beam_decoder,
        maximum_iterations = config.maximum_iterations)

    inter_beam_output_token_idxs = inter_beam_dec_outputs.predicted_ids[:, :, 0]

In [16]:
with tf.variable_scope('sent/dec/rnn', reuse=True):
    topic_dec_initial_state = tf.layers.Dense(units=config.dim_hidden, activation=tf.nn.relu, name='init_state')(means_topic)
    topic_beam_dec_initial_state = tf.contrib.seq2seq.tile_batch(topic_dec_initial_state, multiplier=config.beam_width)
    topic_beam_latents_input = tf.contrib.seq2seq.tile_batch(means_topic, multiplier=config.beam_width) # added
    
    topic_beam_decoder = BeamSearchDecoder(
        cell=dec_cell,
        embedding=embeddings,
        start_tokens=start_tokens,
        end_token=end_token,
        initial_state=topic_beam_dec_initial_state,
        beam_width=config.beam_width, 
        output_layer=output_layer,
        length_penalty_weight=config.length_penalty_weight,
        latents_input=topic_beam_latents_input)

    topic_beam_dec_outputs, _, _ = tf.contrib.seq2seq.dynamic_decode(
        topic_beam_decoder,
        maximum_iterations = config.maximum_iterations)

    topic_beam_output_token_idxs = topic_beam_dec_outputs.predicted_ids[:, :, 0]

In [17]:
with tf.variable_scope('sent/dec/rnn', reuse=True):
    means_topic_summary = tf.reduce_mean(means_topic_infer, 0)
    
    summary_dec_initial_state = tf.layers.Dense(units=config.dim_hidden, activation=tf.nn.relu, name='init_state')(means_topic_summary)
    summary_beam_dec_initial_state = tf.contrib.seq2seq.tile_batch(summary_dec_initial_state, multiplier=config.beam_width)
    summary_beam_latents_input = tf.contrib.seq2seq.tile_batch(means_topic_summary, multiplier=config.beam_width) # added
    
    summary_beam_decoder = BeamSearchDecoder(
        cell=dec_cell,
        embedding=embeddings,
        start_tokens=start_tokens,
        end_token=end_token,
        initial_state=summary_beam_dec_initial_state,
        beam_width=config.beam_width,
        output_layer=output_layer,
        length_penalty_weight=config.length_penalty_weight,
        latents_input=summary_beam_latents_input)

    summary_beam_dec_outputs, _, _ = tf.contrib.seq2seq.dynamic_decode(
        summary_beam_decoder,
        maximum_iterations = config.maximum_iterations)

    summary_beam_output_token_idxs = summary_beam_dec_outputs.predicted_ids[:, :, 0]

## language modeling cost

In [18]:
# target and mask
dec_target_idxs = t_variables['dec_target_idxs']
dec_mask_tokens = tf.sequence_mask(dec_sent_l, maxlen=max_sent_l+1, dtype=tf.float32)

# nll for each token (averaged over batch & sentence)
sent_loss_recon = tf.contrib.seq2seq.sequence_loss(output_logits, dec_target_idxs, dec_mask_tokens)

In [19]:
doc_l = t_variables['doc_l']
mask_sents = tf.sequence_mask(doc_l)
mask_sents_flatten = tf.reshape(mask_sents, [tf.shape(mask_sents)[0]*tf.shape(mask_sents)[1]])

prob_topic_tiled = tf.tile(tf.expand_dims(prob_topic, 1), [1, tf.shape(mask_sents)[1], 1])
prob_topic_flatten = tf.reshape(prob_topic_tiled, [tf.shape(mask_sents)[0]*tf.shape(mask_sents)[1], config.n_topic])
prob_topic_sents = tf.boolean_mask(prob_topic_flatten, mask_sents_flatten)

In [20]:
# inferred mixture probabilities (computed for each sentence)
categ_topic_infer = tfd.Categorical(probs=prob_topic_infer)

# prior of mixture probabilities (computed for each document, tiled for each sentence)
categ_topic = tfd.Categorical(probs=prob_topic_sents)

sent_loss_kl_categ = tf.reduce_mean(tfd.kl_divergence(categ_topic_infer, categ_topic))

# inference of each gaussian gaussribution (computed for each sentence)

sent_loss_kl_gauss = tf.reduce_sum(tfd.kl_divergence(gauss_topic_infer, gauss_topic), -1)
sent_loss_kl_gmm = tf.reduce_mean(tf.reduce_sum(tf.multiply(prob_topic_infer, sent_loss_kl_gauss), -1))

sent_loss_kl = sent_loss_kl_categ + sent_loss_kl_gmm

## optimizer

In [21]:
global_step = tf.Variable(0, name='global_step',trainable=False)
tau = tf.cast(tf.divide(tf.mod(global_step, tf.constant(config.cycle_steps)), tf.constant(config.cycle_steps)), dtype=tf.float32)
beta = tf.minimum(1., tau/config.r_cycle)

sent_loss = sent_loss_recon + beta * sent_loss_kl

topic_loss = topic_loss_recon + topic_loss_kl + config.reg * topic_loss_reg
loss = topic_loss + sent_loss

# define optimizer
if config.opt == 'Adam':
    optimizer = tf.train.AdamOptimizer(config.lr)
elif config.opt == 'Adagrad':
    optimizer = tf.train.AdagradOptimizer(config.lr)
    
grad_vars = optimizer.compute_gradients(loss)
clipped_grad_vars = [(tf.clip_by_value(grad, -config.grad_clip, config.grad_clip), var) for grad, var in grad_vars]

opt = optimizer.apply_gradients(clipped_grad_vars, global_step=global_step)

/home/m-isonuma/.pyenv/versions/anaconda2-5.3.0/envs/py36/lib/python3.6/site-packages/tensorflow/python/ops/gradients_impl.py:112: UserWarning: Converting sparse IndexedSlices to a dense Tensor of unknown shape. This may consume a large amount of memory.
  "Converting sparse IndexedSlices to a dense Tensor of unknown shape. "


# run model 

In [22]:
def idxs_to_sents(token_idxs, config, idx_to_word):
    sents = []
    for sent_idxs in token_idxs:
        tokens = []
        for idx in sent_idxs:
            if idx == config.EOS_IDX: break
            tokens.append(idx_to_word[idx])
        sent = ' '.join(tokens)
        sents.append(sent)
    return sents

In [23]:
def get_loss(sess, batches):
    losses = []
    ppl_list = []
    for ct, batch in batches:
        feed_dict = get_feed_dict(batch, mode='test')
        loss_batch, topic_loss_recon_batch, topic_loss_kl_batch, topic_loss_reg_batch, sent_loss_recon_batch, sent_loss_kl_batch, ppls_batch \
            = sess.run([loss, topic_loss_recon, topic_loss_kl, topic_loss_reg, sent_loss_recon, sent_loss_kl, topic_ppls], feed_dict = feed_dict)
        losses += [[loss_batch, topic_loss_recon_batch, topic_loss_kl_batch, topic_loss_reg_batch, sent_loss_recon_batch, sent_loss_kl_batch]]
        ppl_list += list(ppls_batch)
    loss_mean, topic_loss_recon_mean, topic_loss_kl_mean, topic_loss_reg_mean, sent_loss_recon_mean, sent_loss_kl_mean = np.mean(losses, 0)
    ppl_mean = np.exp(np.mean(ppl_list))
    return loss_mean, topic_loss_recon_mean, topic_loss_kl_mean, topic_loss_reg_mean, sent_loss_recon_mean, sent_loss_kl_mean, ppl_mean

def get_all_losses(sess, batches):
    losses = []
    for ct, batch in batches:
        feed_dict = get_feed_dict(batch, mode='test')
        loss_batch, topic_loss_recon_batch, topic_loss_kl_batch, sent_loss_recon_batch, sent_loss_kl_batch = \
        sess.run([loss, topic_loss_recon, topic_loss_kl, sent_loss_recon, sent_loss_kl], feed_dict = feed_dict)
        losses += [[loss_batch, topic_loss_recon_batch, topic_loss_kl_batch, sent_loss_recon_batch, sent_loss_kl_batch]]
    print('LOSS %.2f | TM NLL: %.2f, KL: %.4f | LM NLL: %.2f, KL: %.4f' %  np.mean(losses, 0))

In [24]:
def print_sample(sample_batch):
    feed_dict = get_feed_dict(sample_batch)
    pred_token_idxs = sess.run(output_token_idxs, feed_dict = feed_dict)
    true_token_idxs = [sent_idxs for instance in sample_batch for sent_idxs in instance.token_idxs]
    
    assert len(pred_token_idxs) == len(true_token_idxs)
    
    pred_sents = idxs_to_sents(pred_token_idxs, config, idx_to_word)
    true_sents = idxs_to_sents(true_token_idxs, config, idx_to_word)
    
    for i, (true_sent, pred_sent) in enumerate(zip(true_sents, pred_sents)):        
        print(i, 'TRUE: %s' % true_sent)
        print(i, 'PRED: %s' % pred_sent)

def print_topic_sample():
    pred_topics_freq_bow_indices, pred_topic_token_idxs = sess.run([topics_freq_bow_indices, topic_beam_output_token_idxs], 
                                                                                                           feed_dict={t_variables['batch_l']: config.n_topic, t_variables['keep_prob']: 1.,})
    pred_topic_sents = idxs_to_sents(pred_topic_token_idxs, config, idx_to_word)
    
    topics_freq_bow_idxs = bow_idxs[pred_topics_freq_bow_indices]
    
    print('-----------Topic Samples-----------')
    for i, (topic_freq_bow_idxs, pred_topic_sent) in enumerate(zip(topics_freq_bow_idxs, pred_topic_sents)):
        print(i, ' BOW:', ' '.join([idx_to_word[idx] for idx in topic_freq_bow_idxs]))
        print(i, ' SENTENCE:', pred_topic_sent)
        
def print_summary(test_batch):
    feed_dict = get_feed_dict(test_batch)
    feed_dict[t_variables['batch_l']] = config.n_topic
    feed_dict[t_variables['keep_prob']] = 1.
    pred_topics_freq_bow_indices, pred_summary_token_idxs = sess.run([topics_freq_bow_indices, summary_beam_output_token_idxs], feed_dict=feed_dict)
    pred_summary_sents = idxs_to_sents(pred_summary_token_idxs, config, idx_to_word)
    
    topics_freq_bow_idxs = bow_idxs[pred_topics_freq_bow_indices]
    
    print('-----------Output sentences for each topic-----------')
    print('Item idx:', test_batch[0].item_idx)
    for i, (topic_freq_bow_idxs, pred_summary_sent) in enumerate(zip(topics_freq_bow_idxs, pred_summary_sents)):
        print(i, ' BOW:', ' '.join([idx_to_word[idx] for idx in topic_freq_bow_idxs]))
        print(i, ' SENTENCE:', pred_summary_sent)
        
    print('-----------Summaries-----------')
    for i, summary in enumerate(test_batch[0].summaries):
        print('SUMMARY %i :'%i, '\n', summary)

In [25]:
if 'sess' in globals(): sess.close()
sess = tf.Session()
sess.run(tf.global_variables_initializer())

losses_train = []
ppls_train = []
loss_min = np.inf
beta_eval = 1.
epoch = 0
train_batches = get_batches(instances_train, config.batch_size, iterator=True)
saver = tf.train.Saver(max_to_keep=10)

log_df = pd.DataFrame(columns=pd.MultiIndex.from_tuples(
                    list(zip(*[['','','','TRAIN:','TM','','','','LM','','VALID:','TM','','','','LM','', ''],
                            ['Time','Ep','Ct','LOSS','PPL','NLL','KL','REG','NLL','KL','LOSS','PPL','NLL','KL','REG','NLL','KL', 'Beta']]))))

In [26]:
if len(log_df) == 0:
    cmd_rm = 'rm -r %s' % config.modeldir
    res = subprocess.call(cmd_rm.split())

    cmd_mk = 'mkdir %s' % config.modeldir
    res = subprocess.call(cmd_mk.split())

time_start = time.time()
while epoch < config.epochs:
    for ct, batch in train_batches:
        feed_dict = get_feed_dict(batch)

        _, loss_batch, topic_loss_recon_batch, topic_loss_kl_batch, topic_loss_reg_batch, sent_loss_recon_batch, sent_loss_kl_batch, sent_loss_kl_categ_batch, sent_loss_kl_gmm_batch, ppls_batch = \
        sess.run([opt, loss, topic_loss_recon, topic_loss_kl, topic_loss_reg, sent_loss_recon, sent_loss_kl, sent_loss_kl_categ, sent_loss_kl_gmm, topic_ppls], feed_dict = feed_dict)
   
        if sent_loss_kl_batch == np.inf:
            print('Nan occured')
            ckpt = tf.train.get_checkpoint_state(config.modeldir)
            model_checkpoint_path = ckpt.all_model_checkpoint_paths[-1]
            saver.restore(sess, model_checkpoint_path)            
            break
            
        losses_train += [[loss_batch, topic_loss_recon_batch, topic_loss_kl_batch, topic_loss_reg_batch, sent_loss_recon_batch, sent_loss_kl_batch]]
        ppls_train += list(ppls_batch)

        if ct%config.log_period==0:
            loss_train, topic_loss_recon_train, topic_loss_kl_train, topic_loss_reg_train, sent_loss_recon_train, sent_loss_kl_train = np.mean(losses_train, 0)
            ppl_train = np.exp(np.mean(ppls_train))
            loss_dev, topic_loss_recon_dev, topic_loss_kl_dev, topic_loss_reg_dev, sent_loss_recon_dev, sent_loss_kl_dev, ppl_dev = get_loss(sess, dev_batches)
            global_step_log, beta_eval = sess.run([tf.train.get_global_step(), beta])
            
            if loss_dev < loss_min:
                loss_min = loss_dev
                saver.save(sess, config.modelpath, global_step=global_step_log)

            clear_output()
    
            time_log = int(time.time() - time_start)
            log_series = pd.Series([time_log, epoch, ct, \
                    '%.2f'%loss_train, '%.0f'%ppl_train, '%.2f'%topic_loss_recon_train, '%.2f'%topic_loss_kl_train, '%.2f'%topic_loss_reg_train, '%.2f'%sent_loss_recon_train, '%.2f'%sent_loss_kl_train, \
                    '%.2f'%loss_dev, '%.0f'%ppl_dev, '%.2f'%topic_loss_recon_dev, '%.2f'%topic_loss_kl_dev, '%.2f'%topic_loss_reg_dev, '%.2f'%sent_loss_recon_dev, '%.2f'%sent_loss_kl_dev,  '%.3f'%beta_eval],
                    index=log_df.columns)
            log_df.loc[global_step_log] = log_series
            display(log_df)

            print_summary(test_batches[1][1])
            print_sample(batch)
            
            time_start = time.time()
            
    epoch += 1
    train_batches = get_batches(instances_train, config.batch_size, iterator=True)

TRAIN:    TM                        LM        VALID:  \
       Time  Ep    Ct    LOSS   PPL     NLL    KL   REG   NLL    KL    LOSS   
1        17   0     0  133.01  1035  122.48  0.42  0.98  9.13  0.80  126.55   
501      72   0   500  121.21   614  114.07  0.05  0.71  6.34  0.98  111.72   
1001     71   0  1000  120.42   584  113.63  0.09  0.57  6.07  0.78  111.25   
1501     71   0  1500  119.95   570  113.25  0.16  0.51  5.96  0.66  111.21   
2001     81   0  2000  119.55   560  112.89  0.25  0.46  5.88  0.59  110.76   
2276     44   1     0  119.40   554  112.75  0.29  0.43  5.84  0.56  110.46   
2776     72   1   500  119.02   547  112.39  0.37  0.39  5.76  0.51  110.20   
3276     70   1  1000  118.66   539  112.05  0.45  0.36  5.69  0.48  109.85   
3776     70   1  1500  118.51   534  111.92  0.52  0.33  5.63  0.45  109.62   
4276     72   1  2000  118.46   528  111.88  0.57  0.31  5.57  0.43  109.48   
4551     45   2     0  118.33   525  111.76  0.61  0.30  5.54  0.42  109.41   
5051     70   2   500  118.18   521  111.61  0.66  0.28  5.49  0.40  109.21   
5551     71   2  1000  118.00   516  111.43  0.71  0.27  5.45  0.39  109.08   
6051     59   2  1500  117.82   512  111.24  0.76  0.25  5.41  0.38  109.19   
6551     72   2  2000  117.75   508  111.16  0.81  0.24  5.37  0.38  108.88   
6826     44   3     0  117.67   506  111.08  0.83  0.23  5.35  0.37  108.87   
7326     70   3   500  117.55   503  110.96  0.87  0.22  5.32  0.37  108.85   
7826     71   3  1000  117.40   500  110.81  0.91  0.21  5.29  0.36  108.60   
8326     78   3  1500  117.32   497  110.73  0.94  0.20  5.26  0.36  108.39   
8826     55   3  2000  117.24   494  110.65  0.97  0.19  5.23  0.35  108.60   
9101     30   4     0  117.20   493  110.61  0.99  0.19  5.22  0.35  108.54   
9601     53   4   500  117.14   490  110.55  1.02  0.18  5.19  0.35  108.46   
10101    66   4  1000  117.06   488  110.47  1.05  0.17  5.17  0.35  108.30   
10601    58   4  1500  117.00   486  110.40  1.07  0.17  5.14  0.34  108.36   
11101    81   4  2000  116.90   484  110.30  1.10  0.16  5.12  0.34  108.22   
11376    41   5     0  116.86   483  110.26  1.11  0.16  5.11  0.34  107.76   
11876    53   5   500  116.76   481  110.16  1.14  0.15  5.09  0.35  107.83   
12376    65   5  1000  116.67   479  110.08  1.16  0.15  5.07  0.36  107.76   
12876    65   5  1500  116.61   477  110.03  1.19  0.14  5.05  0.37  107.68   
13376    53   5  2000  116.55   475  109.97  1.21  0.14  5.03  0.38  107.89   
...     ...  ..   ...     ...   ...     ...   ...   ...   ...   ...     ...   
100101   30  44     0  113.59   396  106.87  2.45  0.03  3.83  0.61  105.81   
100601   54  44   500  113.59   396  106.86  2.46  0.03  3.82  0.61  105.73   
101101   54  44  1000  113.58   395  106.86  2.46  0.03  3.82  0.61  105.84   
101601   53  44  1500  113.58   395  106.85  2.46  0.03  3.81  0.61  105.68   
102101   53  44  2000  113.57   395  106.85  2.46  0.03  3.81  0.61  105.87   
102376   42  45     0  113.57   395  106.84  2.46  0.03  3.81  0.61  105.03   
102876   54  45   500  113.56   395  106.84  2.46  0.03  3.81  0.61  105.03   
103376   54  45  1000  113.55   395  106.83  2.47  0.03  3.80  0.62  105.10   
103876   54  45  1500  113.54   395  106.83  2.47  0.03  3.80  0.62  105.14   
104376   54  45  2000  113.53   395  106.82  2.47  0.03  3.79  0.62  105.30   
104651   31  46     0  113.53   395  106.82  2.47  0.03  3.79  0.62  105.28   
105151   54  46   500  113.52   395  106.81  2.47  0.03  3.79  0.62  105.42   
105651   53  46  1000  113.51   394  106.81  2.48  0.03  3.78  0.62  105.41   
106151   55  46  1500  113.50   394  106.80  2.48  0.03  3.78  0.62  105.56   
106651   55  46  2000  113.50   394  106.80  2.48  0.03  3.78  0.62  105.67   
106926   31  47     0  113.50   394  106.80  2.48  0.03  3.78  0.62  105.69   
107426   54  47   500  113.49   394  106.79  2.48  0.03  3.77  0.62  105.92   
107926   54  47  1000  113.49   394  106.79  2.48  0.03  3.77  0.62  105.

-----------Output sentences for each topic-----------
Item idx: B000VB7EFW
0  BOW: nice feel bit works 'm pro air protect put feels
0  SENTENCE: the soft material , soft , soft , smooth , and feels durable , and feels durable , so far , feels durable and easy to clean and easy to clean , durable , durable , and feels durable and
1  BOW: perfectly protection sleeve protect chromebook soft perfect bought snug inch
1  SENTENCE: this case fits my macbook pro perfectly perfectly with my samsung chromebook perfectly , fits perfectly with my mac air perfectly , protects the chromebook perfectly , protects the chromebook perfectly , protects the chromebook perfectly and protects the chromebook
2  BOW: print surprise saved nicer comments personally ugly user wont realized
2  SENTENCE: this item was a $ # . # and the price was right away from the product description to the description
3  BOW: cover easily hard shell apple logo scratches nice easy rubberized
3  SENTENCE: the rubber feet on the bo

# confirm variables

In [27]:
_prob_topic, _prob_topic_sents, _prob_topic_infer, _means_topic_infer = debug_value([prob_topic, prob_topic_sents, prob_topic_infer, means_topic_infer], return_value=True)

In [28]:
batch_i = 4
_prob_topic_sents[batch_i], _prob_topic_infer[batch_i]

(array([0.02792162, 0.01464365, 0.0005151 , 0.00252109, 0.04934236,
        0.00070111, 0.01627774, 0.00029658, 0.00991118, 0.02318782,
        0.00018818, 0.0047924 , 0.00556466, 0.01821583, 0.00063281,
        0.0041016 , 0.03139242, 0.00631432, 0.00920366, 0.02967596,
        0.00700531, 0.01664687, 0.00617395, 0.00841429, 0.01440318,
        0.07624064, 0.00418605, 0.0311865 , 0.0020076 , 0.01217666,
        0.00486961, 0.07091897, 0.00025259, 0.07629693, 0.00262747,
        0.01475961, 0.05211972, 0.00855024, 0.00333116, 0.00067762,
        0.02526877, 0.03176381, 0.04822835, 0.07904603, 0.03668637,
        0.01281309, 0.01135783, 0.08260436, 0.00019155, 0.00379475],
       dtype=float32),
 array([2.1000540e-02, 4.3415143e-03, 2.3385076e-04, 1.4621429e-03,
        7.4650727e-02, 4.5905221e-04, 1.7114475e-02, 2.0538093e-04,
        3.4309898e-03, 1.9077362e-02, 7.0506288e-04, 3.3221017e-03,
        3.2677110e-03, 6.0324292e-03, 9.5951243e-04, 7.2797793e-03,
        4.9556013e-02, 1

In [29]:
_means_topic_infer[0][:, :4]

array([[-0.77705145, -1.4867538 ,  0.45617026, -0.7698751 ],
       [ 2.272597  , -1.779705  ,  0.67984253, -1.7813202 ],
       [-0.4757314 , -0.71794784,  1.2470813 ,  0.3362029 ],
       [-2.179429  , -2.423539  ,  2.2410502 , -0.13183467],
       [-0.7224728 ,  0.25821558, -0.9037421 ,  1.0290488 ],
       [-0.44092745, -0.74774486,  1.2310039 ,  0.36331385],
       [-0.9691905 , -0.94298947,  0.17719796,  0.31528753],
       [-0.3983578 , -0.5914751 ,  1.1418928 ,  0.36395875],
       [14.998835  , 19.94313   , -6.1085777 , -6.024583  ],
       [-0.6340552 ,  1.5837287 , -0.09692126, -1.7177402 ],
       [ 0.07968902, -0.26739648,  1.0126684 ,  0.2657051 ],
       [17.479574  , 23.29396   , -7.136926  , -6.938561  ],
       [ 0.16300717, -2.5457444 ,  2.4619877 , -0.39026064],
       [ 1.5104747 , -2.2272413 ,  1.0863245 , -2.0352528 ],
       [-2.9354844 , -3.0115507 ,  3.5118918 ,  0.04232958],
       [-1.4653643 , -0.9599113 ,  0.9310325 ,  0.6793495 ],
       [-2.683891  , -1.

In [30]:
w_means_topic, b_means_topic = tf.get_collection(tf.GraphKeys.TRAINABLE_VARIABLES, "topic/dec/mean_topic")

pred_topic_embeddings, pred_topic_bow, pred_means_topic, pred_logvars_topic, pred_token_idxs, _w_means_topic, _b_means_topic, _w_mean_topic_infer = \
                                sess.run([topic_embeddings, topic_bow, means_topic, logvars_topic, topic_beam_output_token_idxs, w_means_topic, b_means_topic, w_mean_topic_infer], 
                                         feed_dict={t_variables['batch_l']: config.n_topic, t_variables['keep_prob']: 1.,})

pred_sents = idxs_to_sents(pred_token_idxs, config, idx_to_word)

pred_topics_freq_bow_indices = np.argsort(pred_topic_bow, 1)[:, ::-1][:, :10]
pred_topics_freq_bow_idxs = bow_idxs[pred_topics_freq_bow_indices]

In [31]:
for idxs in pred_topics_freq_bow_idxs:
    print([idx_to_word[idx] for idx in idxs])

['nice', 'feel', 'bit', 'works', 'pro', 'put', 'air', "'m", 'protect', 'lot']
['perfectly', 'protection', 'sleeve', 'protect', 'bought', 'perfect', 'chromebook', 'inside', 'inch', 'soft']
['print', 'surprise', 'saved', 'nicer', 'personally', 'comments', 'wont', 'ugly', 'user', 'fair']
['cover', 'easily', 'apple', 'shell', 'hard', 'logo', 'nice', 'scratches', 'rubberized', 'easy']
['back', 'carry', 'books', 'work', 'heavy', 'school', 'years', "'ve", 'pack', 'bags']
['neat', 'plain', 'lose', 'user', 'nicer', 'personally', 'fair', 'listed', 'photos', 'hit']
['compartment', 'main', 'pocket', 'small', 'front', 'side', 'access', 'hold', 'bottle', 'water']
['local', 'searched', 'fair', 'user', 'lose', 'nicer', 'surprise', 'realized', 'wont', 'comments']
['!', 'cute', 'super', 'beautiful', 'awesome', 'totally', 'compliments', 'unique', 'absolutely', 'wonderful']
['bought', 'buy', '...', 'recommend', 'purchase', 'time', 'worth', "'m", 'happy', 'money']
['cheaply', 'wont', 'poorly', 'realized', 

In [32]:
pred_topic_embeddings[:, :10]

array([[-3.92404228e-01,  1.35630518e-01,  5.12800477e-02,
         1.03478752e-01, -4.03331339e-01,  1.75541773e-01,
         1.34249315e-01, -1.34838015e-01,  1.26896665e-01,
        -1.38889745e-01],
       [-4.30931538e-01, -6.72928691e-02, -5.09441718e-02,
         9.13887378e-03, -2.40449548e-01,  8.15488696e-02,
        -1.36742562e-01, -2.07162485e-01,  1.10992536e-01,
        -1.09735683e-01],
       [ 1.22895226e-01,  1.78709701e-01,  6.87819161e-03,
        -4.94722337e-01, -3.33423615e-01,  1.99662253e-01,
        -1.32985845e-01, -5.23654819e-02,  7.04008043e-01,
         4.42525029e-01],
       [ 1.58089206e-01, -5.11043631e-02, -1.69768795e-01,
         1.66817848e-02,  4.21261102e-01,  3.02397227e-03,
         2.62937307e-01, -7.98380375e-03, -8.58561173e-02,
        -1.37656063e-01],
       [ 1.73268333e-01, -1.27717219e-02,  5.71838878e-02,
         1.28342614e-01,  2.60495245e-02,  4.26307961e-05,
         9.31888148e-02,  2.89289486e-02, -1.99505210e-01,
         2.

In [33]:
pred_topic_bow

array([[3.21909203e-04, 4.31783177e-04, 1.63430566e-04, ...,
        1.38849282e-06, 1.36577855e-05, 2.89274203e-06],
       [2.24819145e-04, 4.36413247e-05, 5.89604788e-05, ...,
        1.33266058e-05, 1.01355006e-04, 1.80296411e-05],
       [1.72764412e-05, 7.27603910e-05, 7.25599239e-05, ...,
        7.69662165e-06, 1.10924339e-06, 3.18973216e-05],
       ...,
       [1.34782196e-04, 1.30009221e-05, 3.45027584e-05, ...,
        4.39123774e-04, 1.09823566e-04, 1.07166161e-04],
       [2.22593117e-05, 2.99593929e-04, 5.23408758e-04, ...,
        3.85788184e-07, 2.65403116e-07, 9.07258311e-07],
       [9.86009763e-06, 2.63822754e-03, 9.28363879e-04, ...,
        4.59286202e-05, 3.70380498e-04, 2.99739389e-04]], dtype=float32)

In [34]:
_w_means_topic

array([[-0.09029697, -0.3818669 ,  0.08883395, ..., -0.25656465,
        -0.39281362, -0.18671207],
       [ 0.08856896, -0.1262514 ,  0.14311399, ...,  0.001688  ,
         0.06438643, -0.03520157],
       [-0.24564894, -0.12563021, -0.00175388, ..., -0.19886872,
         0.0355957 ,  0.08251901],
       ...,
       [-0.01543219, -0.14567626, -0.0943089 , ..., -0.09193935,
         0.04260384,  0.05946982],
       [ 0.3125463 ,  0.6641227 , -0.12975322, ..., -0.08004636,
        -0.01782492, -0.05968362],
       [-0.03915088,  0.01528641,  0.04972398, ..., -0.07354554,
        -0.07732335, -0.01298761]], dtype=float32)

In [35]:
_b_means_topic

array([-0.11835057, -0.13454448, -0.00337572,  0.05177502,  0.10295276,
        0.02605309, -0.18366031,  0.05934381,  0.1732228 , -0.13531289,
       -0.22492057, -0.12212984,  0.1266292 , -0.25644603,  0.17135005,
        0.09541989, -0.02934371,  0.09172291,  0.04331908, -0.03151641,
       -0.02083609,  0.04228181, -0.0005491 , -0.10296965, -0.17627637,
       -0.18757704, -0.04487782,  0.06314798, -0.18511961,  0.15516071,
        0.05649541, -0.14112212], dtype=float32)

In [36]:
pred_means_topic

array([[-0.76687163, -1.475438  ,  0.43883345, ..., -0.664001  ,
        -2.0241804 , -0.14186974],
       [ 2.2823424 , -1.7608914 ,  0.6601023 , ...,  0.25930965,
        -4.106938  , -0.02110008],
       [-0.46391395, -0.71264696,  1.2170411 , ...,  0.12201966,
         0.16775817, -0.15185839],
       ...,
       [ 0.88815385,  0.49680227, -2.0771904 , ...,  0.29725865,
        -1.712059  , -0.3213429 ],
       [-1.3737322 , -1.8035553 ,  2.3982174 , ...,  0.338076  ,
         0.14978433, -0.19695844],
       [-0.7402571 , -0.86499304,  1.7953227 , ...,  0.5729351 ,
         0.15811875, -0.28681582]], dtype=float32)

In [37]:
_w_mean_topic_infer[:, :10, 0]

array([[-4.64181136e-03,  8.20039306e-03, -5.96803380e-04,
        -3.88584211e-02, -1.42097217e-03,  8.56163539e-03,
        -7.66706653e-03,  1.61374628e-04,  8.14735238e-03,
        -4.56584152e-03],
       [ 1.93836596e-02, -4.16404977e-02,  1.03566749e-02,
         1.00123785e-01,  1.06822690e-02, -1.79884192e-02,
         2.48028599e-02, -2.62945611e-03, -2.12644991e-02,
         3.45119052e-02],
       [-9.00319591e-03,  1.09295929e-02,  6.26969908e-04,
        -1.80125087e-02,  3.42475693e-03,  1.58313420e-02,
         2.54083192e-03,  1.11510800e-02,  1.42021372e-03,
        -9.55972914e-03],
       [-2.21229624e-02,  4.82514352e-02, -9.89297125e-03,
        -7.39470571e-02, -1.59238242e-02,  2.11052559e-02,
        -2.00824663e-02, -6.25270186e-03,  2.71893814e-02,
        -3.04618180e-02],
       [-8.88055749e-03,  2.28155311e-02, -1.15662245e-02,
        -1.88281313e-02, -1.12704057e-02,  5.43615967e-03,
        -1.48752062e-02,  3.10412422e-03,  6.55456074e-03,
        -1.

In [38]:
_b_means_topic

array([-0.11835057, -0.13454448, -0.00337572,  0.05177502,  0.10295276,
        0.02605309, -0.18366031,  0.05934381,  0.1732228 , -0.13531289,
       -0.22492057, -0.12212984,  0.1266292 , -0.25644603,  0.17135005,
        0.09541989, -0.02934371,  0.09172291,  0.04331908, -0.03151641,
       -0.02083609,  0.04228181, -0.0005491 , -0.10296965, -0.17627637,
       -0.18757704, -0.04487782,  0.06314798, -0.18511961,  0.15516071,
        0.05649541, -0.14112212], dtype=float32)

In [39]:
_enc_state_infer, _means_topic_infer = debug_value([enc_state_infer, means_topic_infer], return_value=True)

NameError: name 'enc_state_infer' is not defined

In [ ]:
_enc_state_infer.shape

In [ ]:
_means_topic_infer[0]